In [1]:
#coding utf-8

import numpy as np
from sklearn.decomposition import PCA
from mayavi import mlab


def create_ellipsoid_surface(abc, theta, phi, center):
	x = abc[0] * np.sin(theta) * np.cos(phi)	+ center[0]
	y = abc[1] * np.sin(theta) * np.sin(phi)	+ center[1]
	z = abc[2] * np.cos(theta) 					+ center[2]

	return np.array([x, y, z])


def from_spherical_to_xyz(r, theta, phi):
	return create_ellipsoid_surface([r, r, r], theta, phi, [0,0,0])


def create_dust(N):

	balls_arr = np.empty(shape=(N,4))

	r = np.random.uniform(0, 1)
	balls_arr[0, :] = [0, 0, 0, r]

	for i in range(1, N):

		r = np.random.uniform(0, 1)
		theta = np.random.uniform(0, np.pi)
		phi = np.random.uniform(0, 2*np.pi)

		ind = np.random.randint(0, i)
		alpha = np.random.uniform(0, 1)
		d = balls_arr[ind, -1] + alpha*r

		xyz = from_spherical_to_xyz(d, theta, phi)
		xyz += balls_arr[ind, :-1]

		balls_arr[i, :] = np.concatenate((xyz, np.array([r])))

	return balls_arr


def approx_fun(dots_arr, sigma):

	pca = PCA(n_components=3)
	# XPCAreduced = pca.fit_transform(balls_arr[:,:-1])
	XPCAreduced = pca.fit_transform(dots_arr)

	# sigma = 3

	dlina = np.sqrt(pca.explained_variance_)*sigma
	center = pca.mean_
	QT = pca.components_

	kj = 20j
	u, v = np.mgrid[0:2*np.pi:kj, 0:np.pi:kj]
	k = len(u)

	x, y, z = create_ellipsoid_surface(dlina, v, u, [0,0,0])

	x1 = np.empty([k,k])
	y1 = np.empty([k,k])
	z1 = np.empty([k,k])

	for i in range(k):
		for j in range(k):
			x1[i,j],y1[i,j],z1[i,j] = QT.T @ np.array([x[i,j],y[i,j],z[i,j]]) + center

	return [x1, y1, z1, dlina]


def from_balls_to_dots(balls_arr):
	dots_arr = balls_arr[:,:-1]

	kj = 50j
	u, v = np.mgrid[0:2*np.pi:kj, 0:np.pi:kj]
	for ball in balls_arr:
		r = ball[-1]

		x, y, z = create_ellipsoid_surface([r, r, r], v, u, ball[:-1])

		x = np.asarray(x).reshape(-1)
		y = np.asarray(y).reshape(-1)
		z = np.asarray(z).reshape(-1)

		d = np.dstack([x,y,z])[0]

		new_d = [tuple(row) for row in d]
		uniques = np.unique(new_d, axis=0)

		dots_arr = np.append(dots_arr, uniques, axis=0)

	return dots_arr


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [2]:
N = 50

balls_arr = create_dust(N)
dots_arr = from_balls_to_dots(balls_arr)

In [3]:
n_sigma = 3
x1, y1, z1, abc = approx_fun(dots_arr, n_sigma)


l = 2*abc[0]

kj = 50j
u, v = np.mgrid[0:2*np.pi:kj, 0:np.pi:kj]

for ball in balls_arr:
	r = ball[-1]

	x, y, z = create_ellipsoid_surface([r, r, r], v, u, ball[:-1])
	
	mlab.mesh(x, y, z, colormap="OrRd")
	mlab.mesh(x+l, y+l, z+l, colormap="OrRd")

mlab.mesh(x1+l, y1+l, z1+l, representation='wireframe', color=(0.09, 0.01, 0.65))

print("Semi-axes of the approximating ellipsoid:")
print(f"a = {abc[0]:.4f}")
print(f"b = {abc[1]:.4f}")
print(f"c = {abc[2]:.4f}")

mlab.show()

Semi-axes of the approximating ellipsoid:
a = 2.3148
b = 1.3341
c = 1.2827
